In [26]:
from linopy import *
import pandas as pd
import xarray as xr
from linopy import Model
from matplotlib import cm
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### Data
- import data
- print the sheet names without looking at the file

In [27]:

data = pd.ExcelFile("./OverviewM2.xlsx")
data.sheet_names

['Simulation',
 'Input_powerplants',
 'Input_demand',
 'Input_RES',
 'Input_RES_2',
 'Input_Storage',
 'Input_Grid']

### Create a data set
- select the sheet name that contains the necessary data
- print the dataframe
- select the correct range of the data
  

In [28]:
inpp = pd.read_excel(data, sheet_name='Input_powerplants', skiprows=1) # (inpp - Input Power Plants)

In [29]:
col1 = inpp.iloc[1, 0:2]
col2 = inpp.iloc[0,2:]
inpp = inpp.iloc[2:9, 0:]
inpp.columns = pd.concat([col1, col2], axis=0, ignore_index=True)
inpp = inpp.loc[inpp.index.repeat(inpp['N_UNITS'])].drop(['N_UNITS'], axis =1)
inpp['Count'] = inpp.groupby('Technology').cumcount() + 1
inpp['Technology'] = inpp['Technology'] + inpp['Count'].astype(str)
inpp.set_index('Technology', inplace=True)

In [30]:
inpp 


,Fuel,Node,GEN_MAX,EFF,GEN_MIN,EFF_PMIN,FUEL_COST,CO2_INT,DELTA_MAX_UP,DELTA_MAX_DOWN,...,MINDOWN,STC2,STC_IND,RC,STC,C,MA,D,MB,Count
Technology,,,,,,,,,,,,,,,,,,,,,
Nuclear1,UO2,1,1200,33,600,30,2,0,450,450,...,32,35,0,0,0,4000,5.454545,0,0,1
Nuclear2,UO2,1,1200,33,600,30,2,0,450,450,...,32,35,0,0,0,4000,5.454545,0,0,2
Nuclear3,UO2,1,1200,33,600,30,2,0,450,450,...,32,35,0,0,0,4000,5.454545,0,0,3
ConventionalC1,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,1
ConventionalC2,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,2
ConventionalC3,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,3
ConventionalC4,Coal,1,800,40,344,37.299296,12,0.3384,240,240,...,12,25,55,1.8,0,11067.233078,28.361331,312.095948,0.799789,4
CCGT1,Gas,1,450,55,157.5,43.451957,25,0.2052,405,405,...,4,5,40,0.5,0,9061.732325,38.949788,74.378693,0.3197,1
CCGT2,Gas,1,450,55,157.5,43.451957,25,0.2052,405,405,...,4,5,40,0.5,0,9061.732325,38.949788,74.378693,0.3197,2


In [31]:
inpp =inpp.to_xarray()    #  inpp = inpp.set_coords(['Technology', 'Fuel'])
inpp

<xarray.Dataset>
Dimensions:         (Technology: 23)
Coordinates:
  * Technology      (Technology) object 'Nuclear1' 'Nuclear2' ... 'GT4' 'GT5'
Data variables: (12/21)
    Fuel            (Technology) object 'UO2' 'UO2' 'UO2' ... 'Gas' 'Gas' 'Gas'
    Node            (Technology) object 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    GEN_MAX         (Technology) object 1200 1200 1200 800 800 ... 50 50 50 50
    EFF             (Technology) object 33 33 33 40 40 40 ... 55 40 40 40 40 40
    GEN_MIN         (Technology) object 600 600 600 344 344 ... 15 15 15 15 15
    EFF_PMIN        (Technology) object 30 30 ... 25.55232634808877
    ...              ...
    STC             (Technology) object 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    C               (Technology) object 4000 4000 ... 1467.5767477744691
    MA              (Technology) object 5.454545454545455 ... 47.35495006358659
    D               (Technology) object 0 0 ... 12.045868982063324
    MB              (Technology) object 0 0 ... 0.38868939902676675
    Count           (Technology) int64 1 2 3 1 2 3 4 1 2 ... 8 9 10 11 1 2 3 4 5

### Data Demand

In [32]:
inpd = pd.read_excel(data, sheet_name='Input_demand', skiprows=3)
inpd = inpd.set_index(['day', 'month', 'year'])
inpd

,,,0:15,0:30,0:45,1:00,1:15,1:30,1:45,2:00,2:15,2:30,...,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45,24:00
day,month,year,,,,,,,,,,,,,,,,,,,,,
1,6.0,2013.0,8673.042,8429.322,8308.632,8140.671,8006.779,7807.729,7763.125,7634.726,7587.030,7482.310,...,7982.102,7988.921,8209.316,8370.824,8393.274,8380.822,8503.550,8353.015,8281.022,8177.031
2,6.0,2013.0,8052.679,7856.342,7743.303,7650.331,7528.002,7468.452,7255.969,7101.194,7034.424,7089.714,...,7739.691,7767.356,7966.283,8354.370,8369.891,8299.814,8297.342,8101.211,7764.114,7706.652
3,6.0,2013.0,7550.928,7438.698,7289.953,7176.891,7165.338,7130.795,6999.951,6880.288,6834.142,6766.567,...,8808.267,8773.263,9024.721,9411.377,9328.347,9108.317,9205.300,9041.506,8697.893,8637.049
4,6.0,2013.0,8496.884,8319.899,8196.701,7967.601,8032.783,7950.501,7928.243,7805.097,7665.461,7584.409,...,8881.317,8911.268,9190.812,9582.027,9560.055,9529.220,9546.534,9463.755,9171.279,8992.793
5,6.0,2013.0,8780.559,8687.634,8394.841,8200.335,8207.454,8095.371,7935.834,7898.115,7727.304,7793.651,...,8949.598,8914.767,9288.891,9542.090,9488.956,9440.409,9344.824,9135.563,8819.755,8710.935
6,6.0,2013.0,8530.443,8441.718,8199.603,8210.283,8023.118,7965.610,7853.734,7718.865,7712.707,7557.747,...,8738.274,8642.352,8920.258,9259.679,9206.432,9132.940,8948.530,8916.491,8732.527,8615.033
7,6.0,2013.0,8429.832,8271.792,8121.208,8134.763,8045.496,7884.031,7740.750,7558.598,7679.048,7523.200,...,8723.628,8692.391,8832.275,9265.558,9255.529,9182.997,9111.409,9003.198,8871.769,8587.459
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nodal division of demand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
demand = inpd.loc[(1,6,2013)].values.flatten().tolist() + inpd.loc[(2,6,2013), '0:15': '12:00'].values.flatten().tolist()

steps = pd.Index(range(len(demand)), name = 'steps')
demand = xr.DataArray(demand, coords= [steps])


In [34]:
RE = pd.read_excel(data, sheet_name='Input_RES_2')
RE.set_index(steps, inplace=True)
GSOLAR = RE['GSOLAR'].fillna(0)
GWIND = RE['GWIND'].fillna(0)

### Storage

In [35]:
inps = pd.read_excel(data, sheet_name='Input_Storage', skiprows=2)
inps.drop(inps.index[0], inplace=True)
inps.rename(columns={'Unnamed: 0': 'Storage'}, inplace=True)
inps.set_index('Storage', inplace=True)
inps

,Node,Pmax,Rated efficiency (pumping),Rated efficiency (turbining),Emax,Emin
Storage,,,,,,
1,3,600,80,80,6000,600
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
Number of storage units,1,NaN,NaN,NaN,NaN,NaN


In [36]:
inps_set = inps.head(1).to_xarray()
inps_set

<xarray.Dataset>
Dimensions:                       (Storage: 1)
Coordinates:
  * Storage                       (Storage) object 1
Data variables:
    Node                          (Storage) object 3
    Pmax                          (Storage) object 600
    Rated efficiency (pumping)    (Storage) object 80
    Rated efficiency (turbining)  (Storage) object 80
    Emax                          (Storage) object 6000
    Emin                          (Storage) object 600

In [37]:
gp = pd.read_excel(data, sheet_name='Simulation', skiprows=1)
gp = gp.drop(gp.columns[1], axis =1).set_index('Simulation_parameters').to_xarray()

In [38]:
gp

<xarray.Dataset>
Dimensions:                (Simulation_parameters: 20)
Coordinates:
  * Simulation_parameters  (Simulation_parameters) object 'Session' ... 'rins...
Data variables:
    Value                  (Simulation_parameters) object 'Basic' 1 ... 0 100
    Unnamed: 3             (Simulation_parameters) object "Session: 'Basic','...

In [39]:

G_INIT = inpp.GEN_MAX.loc['Nuclear1':'CCGT5']
Z_INIT = xr.DataArray(1,coords=[inpp.Technology.loc['Nuclear1':'CCGT5'], steps[:96]])
G_INIT=G_INIT.expand_dims(steps = steps).transpose('Technology', 'steps')

### Global parameters

In [40]:
gp.sel(Simulation_parameters = 'Time step') 
gp.loc[dict(Simulation_parameters = 'Time step')]
gp['Value'].sel(Simulation_parameters = 'Time step')
gp.Value.loc['Time step']

<xarray.DataArray 'Value' ()>
array(0.25, dtype=object)
Coordinates:
    Simulation_parameters  <U9 'Time step'

In [41]:
# def model():
#     m = Model()
#     scu = m.add_variables(coords= [inpp.Technology], name = 'scu')
#     return m.solution.to_dataframe(), m.objective.value

In [42]:
m = Model()
# scu = m.add_variables(lower= 0, coords= [inpp.Technology], name = 'scu')

In [43]:
scu = m.add_variables(lower= 0, coords= [inpp.Technology, steps], name = 'scu')
z = m.add_variables(binary = True, coords= [inpp.Technology, steps], name = 'z')
v = m.add_variables(binary = True, coords= [inpp.Technology, steps], name = 'v')
w = m.add_variables(binary = True, coords= [inpp.Technology, steps], name = 'w')
curtail  = m.add_variables(lower = 0, coords= [steps], name = 'curtail')
fuelcost = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'fuelcost') #fcd in gams
co2cost = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'co2cost') #ccd in gams
rampingcost = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'rampingcost') #rcd in gams
load_shedding = m.add_variables(lower= 0, coords= [steps], name = 'load_shedding')
powergen = m.add_variables(lower = 0, coords= [inpp.Technology, steps], name = 'powergen') #g
pump = m.add_variables( coords= [inps_set.Storage, steps], name = 'pump') #pumping
turbine = m.add_variables( coords= [inps_set.Storage, steps], name = 'turbine')
turbining_power = m.add_variables( coords= [inps_set.Storage, steps], name = 'turbining_power') #pdown in gams
pumping_power = m.add_variables( coords= [inps_set.Storage, steps], name = 'pumping_power') #pup in gams
puct = m.add_variables( coords= [inps_set.Storage, steps], name = 'puct') # energy content hydro storage

In [44]:
total_cost = ((scu + gp.Value.loc['Time step']*fuelcost + gp.Value.loc['Time step']*co2cost+rampingcost).sum(dims = ['Technology', 'steps']) ) + (gp.Value.loc['VOLL']*gp.Value.loc['Time step']*load_shedding).sum(dims ='steps') + (gp.Value.loc['VOC']*gp.Value.loc['Time step']*curtail).sum(dims ='steps')
m.add_objective(total_cost)

In [45]:
# Costs
startup = m.add_constraints(inpp['STC']*v == scu, name = 'startup')
fuel = m.add_constraints( fuelcost >= inpp['C']*z + inpp['MA']*(powergen - inpp['GEN_MIN']*z), name = 'fuel')
co2 = m.add_constraints(co2cost >= gp.Value.loc['CO2_price']*(inpp['D']*z+inpp['MB']*(powergen-inpp['GEN_MIN']*z)), name = 'co2')
rampingcost_pos1 = m.add_constraints(rampingcost.where(steps== 0) >=  inpp['RC']*powergen.where(steps== 0) - inpp['RC']*inpp['GEN_MIN']*v.where(steps==0) , name = 'rampingcost_pos1')
rampingcost_pos2 = m.add_constraints(rampingcost.where(steps>0) >=  inpp['RC']*(powergen.where(steps>0)-powergen.shift(steps = 1)) - inpp['RC']*inpp['GEN_MIN']*v.where(steps>0), name = 'rampingcost_pos2')
rampingcost_neg1 = m.add_constraints((rampingcost.where(steps== 0) >= inpp['RC']*-powergen.where(steps== 0)-inpp['RC']*inpp['GEN_MIN']*w.loc[:,1]), name = 'rampingcost_neg1')
rampingcost_neg2 = m.add_constraints((rampingcost.where(steps>0) >= inpp['RC']*(powergen.shift(steps = 1) - powergen.loc[:, 1:])- inpp['RC']*inpp['GEN_MIN']* w.shift( steps = -1).where(steps>0)), name = 'rampingcost_neg2') 

# Market clearing 
mcc = m.add_constraints(powergen.sum(dims = 'Technology')  + GWIND + GSOLAR - curtail + load_shedding == demand, name = 'mcc')

# Curtailment RES
curt_lim = m.add_constraints(curtail <= GSOLAR + GWIND, name = 'curt_lim')

# load shedding limit
ll_lim = m.add_constraints(load_shedding<=demand, name = 'll_lim')

# Generation limits 
power_max = m.add_constraints(powergen <= inpp['GEN_MAX']*z , name= 'power_max')
power_min = m.add_constraints(powergen >= inpp['GEN_MIN']*z, name = 'power_min')

# Ramping and up/down time constraints 
power_rampup1 = m.add_constraints(powergen.loc[G_INIT['Technology']].where(steps == 0) <= inpp['DELTA_MAX_UP']*z.loc[G_INIT['Technology']].where(steps == 0)+ (inpp['GEN_MIN']-inpp['DELTA_MAX_UP'])*v.loc[G_INIT['Technology']].where(steps == 0) + G_INIT.where(steps ==0), name = 'power_rampup1')
power_rampup2 = m.add_constraints(powergen.where(steps>0) <= powergen.shift(steps = 1) + inpp['DELTA_MAX_UP']*z.where(steps>0)+ (inpp['GEN_MIN']-inpp['DELTA_MAX_UP'])*v.where(steps>0), name = 'power_rampup2')

power_rampdown1 = m.add_constraints(powergen.loc[G_INIT['Technology']].where(steps ==0) >= - inpp['DELTA_MAX_DOWN']*z.loc[G_INIT['Technology']].where(steps ==0) - (inpp['GEN_MIN'])*w.loc[G_INIT['Technology']].where(steps == 0)+ G_INIT.where(steps ==0), name = 'power_rampdown1')
power_rampdown2 = m.add_constraints(powergen.where(steps>0) >= powergen.shift(steps = 1)   - inpp['DELTA_MAX_DOWN']*z.where(steps>0)- (inpp['GEN_MIN'])*w.where(steps>0), name = 'power_rampdown2')

# up_time = m.add_constraints(z >= v.rolling(steps= 144).sum(), name = 'up_time')
# down_time =  m.add_constraints(1 >= z + w.rolling(steps= 144).sum(), name ='down_time')

linking1 = m.add_constraints(-z.where(steps == 0) + Z_INIT.loc[:,0] == w.where(steps == 0)-v.where(steps == 0), name= 'linking1')
linking2= m.add_constraints(z.shift(steps = 1)-z.where(steps > 0) == w.where(steps > 0) - v.where(steps > 0) , name ='linking2')

logic_link = m.add_constraints(w+v <= 1, name = 'logic_link')




In [46]:
m.solve('highs')

Writing constraints.:  16%|█▌        | 3/19 [00:00<00:00, 21.53it/s]

Writing binary variables.: 100%|██████████| 3/3 [00:00<00:00, 98.99it/s]


In [ ]:
m.objective

Objective:
----------
LinearExpression: +1 scu[Nuclear1, 0] + 1 scu[Nuclear1, 1] + 1 scu[Nuclear1, 2] ... +0 curtail[141] + 0 curtail[142] + 0 curtail[143]
Sense: min
Value: 2849853.214556104

In [ ]:
# g(i,j) =l= g(i,j-1) + G_INIT(i)$(ord(j) eq 1)+ DELTA_MAX_UP(i)*z(i,j) + (GEN_MIN(i)-DELTA_MAX_UP(i))*v(i,j); 
# (powergen.loc[G_INIT.loc[:,0],0] <= inpp['DELTA_MAX_UP']*z.loc[G_INIT.loc[:,0],0] + G_INIT.loc[:,0]).print(500)
# m.add_constraints(powergen.loc[G_INIT['Technology'],0] <= inpp['DELTA_MAX_UP']*z.loc[G_INIT['Technology'],0]+ (inpp['GEN_MIN']-inpp['DELTA_MAX_UP'])*v.loc[G_INIT['Technology'],0] + G_INIT)

### selecting rows by loc method

In [ ]:
data.close()